In [1]:
!pip install ultralytics
!pip install -U ipywidgets

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/e0/06/f07c56b8080af737ad42991fc1bb4e94da490c5227225c8337b8c248bd44/ultralytics-8.1.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 756.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.7/699.7 kB 5.8 MB/s eta 0:00:00
  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/4a/0e/57ed498fafbc60419a9332d872e929879ceba2d73cb11d284d7112472b3e/ipywidgets-8.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.9 from https://files.pythonhosted.org/packages/29/03/107d96077c4befed191f7ad1a12c7b52a8f9d2778a5836d59f9855c105f6/widgetsnbextension-4.0.9-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab-widgets~=3.0.9 from https://files.pythonhosted.org/packages/e8/05/0ebab152288693b5ec7b339aab857362947031143b282853b4c2dd4b5b40/jupyterlab_widgets-3.0

In [2]:
!mkdir -p /kaggle/temp/train/labels
!mkdir -p /kaggle/temp/train/images

!mkdir -p /kaggle/temp/val/labels
!mkdir -p /kaggle/temp/val/images

In [3]:
!cp /kaggle/input/kidney-1-dense-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-1-dense-to-yolo/train/images/* /kaggle/temp/train/images

Copy only images 500-899 for test (these are approximately the labels that are available for kidney_3_dense as well)

In [4]:
!cp /kaggle/input/kidney-3-dense-to-yolo/train/labels/kidney_3_dense_0[5-8]*.txt /kaggle/temp/val/labels
!cp /kaggle/input/kidney-3-dense-to-yolo/train/images/kidney_3_dense_0[5-8]*.tif /kaggle/temp/val/images

Explore image sizes

In [5]:
import cv2
import os

sets = ['train','test']
for s in sets:
    dirs = os.listdir(f'/kaggle/input/blood-vessel-segmentation/{s}/')
    for d in dirs:
        if d != 'kidney_3_dense':
            prefix = f'/kaggle/input/blood-vessel-segmentation/{s}/{d}/images/'
            image_file = os.listdir(prefix)[0]
            image_path = f'{prefix}{image_file}'
            print(f'shape for {d} is {cv2.imread(image_path).shape}')

shape for kidney_1_dense is (1303, 912, 3)
shape for kidney_2 is (1041, 1511, 3)
shape for kidney_1_voi is (1928, 1928, 3)
shape for kidney_3_sparse is (1706, 1510, 3)
shape for kidney_5 is (1303, 912, 3)
shape for kidney_6 is (1303, 912, 3)


In [6]:
kidney_1_dense_sample_image_path = '/kaggle/input/blood-vessel-segmentation/train/kidney_1_dense/images/0000.tif'
train_img_size = cv2.imread(kidney_1_dense_sample_image_path).shape[0]

# Write yolo dataset yaml

In [7]:
import yaml

In [8]:
!mkdir train

In [9]:
train_path = '/kaggle/temp/train'
val_path = '/kaggle/temp/val'
names = {0: 'Vessel'}
yaml_path = 'train/train.yaml'

# Number of classes
nc = len(names)

# Create a dictionary with the required content
yaml_data = {
    'names': names,
    'nc': nc,
    'test': '',
    'train': train_path,
    'val': val_path
}

# Write the dictionary to a YAML file
with open(yaml_path, 'w') as file:
    yaml.dump(yaml_data, file, default_flow_style=False)

In [10]:
!cat {yaml_path}

names:
  0: Vessel
nc: 1
test: ''
train: /kaggle/temp/train
val: /kaggle/temp/val


In [11]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("wandb-api-key")
wandb.login(key=wandb_api_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [12]:
from ultralytics import YOLO

model = YOLO('yolov8n-seg.pt')

100%|██████████| 6.73M/6.73M [00:00<00:00, 75.9MB/s]


In [13]:
project = 'hacking_human_vasculature'
name = 'train_kidney_1_dense_val_kidney_3_dense_size_1303'

results = model.train(data=yaml_path,
                      project=project,
                      name=name,
                      epochs=200,
                      time=4,
                      patience=30, 
                      batch=16,
                      imgsz=train_img_size,
                      device=[0, 1],
                      save_period=1,
                      cache=True
                     )

Ultralytics YOLOv8.1.2 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla T4, 15102MiB)
                                                     CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=train/train.yaml, epochs=200, time=4, patience=30, batch=16, imgsz=1303, save=True, save_period=1, cache=True, device=[0, 1], workers=8, project=hacking_human_vasculature, name=train_kidney_1_dense_val_kidney_3_dense_size_1303, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, 

100%|██████████| 755k/755k [00:00<00:00, 13.8MB/s]
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


TensorBoard: Start with 'tensorboard --logdir hacking_human_vasculature/train_kidney_1_dense_val_kidney_3_dense_size_1303', view at http://localhost:6006/


...

      49/65      11.5G     0.7646     0.6364     0.4236     0.7825        325       1312: 100%|██████████| 143/143 [03:38<00:00,  1.53s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:17<00:00,  1.43it/s]


                   all        400      40108      0.522      0.403       0.52      0.274      0.346      0.267      0.303      0.117
Stopping training early as no improvement observed in last 30 epochs. Best results observed at epoch 19, best model saved as best.pt.
To update EarlyStopping(patience=30) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

49 epochs completed in 3.033 hours.
Optimizer stripped from hacking_human_vasculature/train_kidney_1_dense_val_kidney_3_dense_size_1303/weights/last.pt, 6.9MB
Optimizer stripped from hacking_human_vasculature/train_kidney_1_dense_val_kidney_3_dense_size_1303/weights/best.pt, 6.9MB

Validating hacking_human_vasculature/train_kidney_1_dense_val_kidney_3_dense_size_1303/weights/best.pt...
Ultralytics YOLOv8.1.2 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla T4, 15102MiB)
                                                     CUDA:1 (Tesla T4, 15102MiB)
YOLOv8n-seg summary (fused): 195 layers, 3258259 par

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:54<00:00,  2.19s/it]


                   all        400      40108      0.459      0.404      0.496      0.275      0.342      0.301      0.324      0.136
Speed: 3.2ms preprocess, 11.1ms inference, 0.0ms loss, 5.4ms postprocess per image
Results saved to hacking_human_vasculature/train_kidney_1_dense_val_kidney_3_dense_size_1303


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▂▅███▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:                  lr/pg1 ▂▅███▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:                  lr/pg2 ▂▅███▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:        metrics/mAP50(B) ▁▃▃▄▃▄▄▅▃▅▄▆▅▆▅▇▅▆▆▆▇▇▇▇▇▇█▆▇▇▇▇▇▇▇▆▇█▇▇
wandb:        metrics/mAP50(M) ▁▃▆▅▃▃▄▆▁▄▃▆▅▅▄█▄▅▅▅▅▇▇▇█▆▇▅▆▆▇▇▇▆▆▅▇▇▆█
wandb:     metrics/mAP50-95(B) ▁▂▂▄▂▄▂▅▂▅▃▆▄▄▃█▃▅▆▆▆█▇▇█▅█▅▆▆▇▆▆▆▇▄▇█▆█
wandb:     metrics/mAP50-95(M) ▂▃▅▄▃▃▃▆▁��▂▅▃▄▃█▃▅▅▅▅█▆▇▇▅▇▄▅▅▆▅▆▅▆▃▆▆▅█
wandb:    metrics/precision(B) █▆▅▆▅▇▅▆▆▇▆▇▆▆▅▁▅▆▆▆▁▇▆▇▇▆▂▆▆▆▆▂▇▆▆▆▆▂▆▁
wandb:    metrics/precision(M) █▅▅▅▄▆▅▅▄▅▄▅▄▄▄▂▄▄▄▄▁▅▅▅▅▄▂▄▁▄▁▂▄▁▁▄▅▂▅▂
wandb:       metrics/recall(B) ▁▅▆▆▅▅▆▇▅▅▅▅▆▅▅█▅▅▆▆▇▆▆▆▆▆█▅▅▅▆▇▅▅▆▅▆█▆█
wandb:       metrics/recall(M) ▁▄▆▅▅▄▆▇▃▄▄▄▅▄▄█▄▅▅▄▆▅▅▅▅▄▇▄▆▄▇▆▅▇▇▄▅▆▅█
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch

In [14]:
wandb.finish()